In [1]:
import pandas as pd
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

c:\Users\chint\OneDrive\Desktop\RAG-HotelBookings\hotel_bookingsenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
insights = pd.read_csv("C:\\Users\\chint\\OneDrive\\Desktop\\RAG-HotelBookings\\data/insights_df.csv")

In [3]:
insights[insights['type']=='cancellation_rate']

,type,insight
26,cancellation_rate,37.041628277075134


In [4]:
documents = []
for i, row in insights.iterrows():
    doc = Document(page_content=row['insight'], metadata={"type": row['type']})
    documents.append(doc)

print(f"Loaded {len(documents)} documents.")

Loaded 445 documents.


In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Use from_documents for proper metadata handling
vector_store = FAISS.from_documents(documents, embedding_model)

# Save the FAISS index
vector_store.save_local("faiss_insights")
print("FAISS Vector Store Created & Saved!")

C:\Users\chint\AppData\Local\Temp\ipykernel_18040\710582449.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\chint\OneDrive\Desktop\RAG-HotelBookings\hotel_bookingsenv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\chint\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your di

FAISS Vector Store Created & Saved!


In [6]:

# Reload FAISS Index with safe deserialization
vector_store = FAISS.load_local("faiss_insights", embedding_model, allow_dangerous_deserialization=True)


def retrieve_insights(query, k=3, filter_type=None):
    """Retrieves insights from FAISS with optional filtering."""
    results = vector_store.similarity_search_with_score(query, k=k)

    # Applying metadata filtering
    if filter_type:
        results = [(doc, score) for doc, score in results if doc.metadata['type'] == filter_type]

    return results


In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline

# Loading Model & Tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,low_cpu_mem_usage=True,device_map="auto") 

# Creating LLM pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.7)
llm = HuggingFacePipeline(pipeline=pipe)


OSError: The paging file is too small for this operation to complete. (os error 1455)

In [ ]:
def answer_query_with_llm(query, k=3, filter_type=None):
    """Retrieves insights and generates an answer using an LLM."""
    results = retrieve_insights(query, k=k, filter_type=filter_type)

    if not results:
        return "I don't have enough information to answer."

    # Formating retrieved insights for context
    context = "\n".join([f"- {doc.page_content}" for doc, _ in results])

    prompt = f"""
    You are an AI assistant answering user questions based on the provided data.
    Use the context below to answer, and if unsure, respond with 'I don't have enough information to answer.'

    Context:
    {context}

    Question: {query}

    Answer:
    """
    return llm(prompt)

#  Testing the Full RAG System
print("\nTesting RAG system:")
print(answer_query_with_llm("What was the cancellation rate for resort hotels?", k=5, filter_type="cancellation_rate_by_hotel"))
print(answer_query_with_llm("What were the revenue trends last year?", k=5))
print(answer_query_with_llm("Tell me about booking lead times.", k=5))
